In [1]:
def fluctuating_env(step = 0.01):
    init = 0.5
    envs = [init]
    flag = 1
    for i in range(10000):
        envs.append(envs[-1] + step * flag)
        if envs[-1] + step * flag >= 1 or \
            envs[-1] + step * flag <= 0:
                flag = -1 * flag
        envs.append(envs[-1] + step * flag)
    return envs

In [4]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}

settings.MUTATION_PRO = 0.01

class A_6(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_6(GenModel):
    def get_agent_config(self):
        return A_6
    
    def init_env(self):
        return fluctuating_env(self.env_step)
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M5'
        
        init_ratio = 0.5
        self.env_step = specific_kargs['env_step']
        
        settings.LIFE_TIME = specific_kargs['lifetime']
        
         
        settings.INIT_MEAN = 0.5
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_LOW = settings.INIT_MEAN - 1 * 2 * settings.INIT_SD
        settings.INIT_HIGH = settings.INIT_MEAN + 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = settings.INIT_SD
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([k + "_" + str(specific_kargs[k]) for k in specific_kargs])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = check_path("./res/")
        data_file_name = check_path(data_file_name + self.model_name)
        data_file_name = check_path(data_file_name + "/" + self.specific_kargs_str)
        data_file_name = data_file_name + '/' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            ('step', 'env',
             TYPE_MAP[0], TYPE_MAP[0] + '_beta', TYPE_MAP[0] + '_gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + '_beta', TYPE_MAP[1] + '_gene', 'var'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_gene_mean(self, gen_type):
        return [np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type]
    
    def get_popu_gene_mean(self, gene_mean):
        return np.mean(gene_mean)

    def get_popu_gene_var(self, gene_mean):
        return np.var(gene_mean)

    def collect_data(self):
        ver_mean = self.get_gene_mean(0)
        hor_mean = self.get_gene_mean(1)
        return (self.schedule.steps, self.cur_press,
                self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(ver_mean), self.get_popu_gene_var(ver_mean),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(hor_mean), self.get_popu_gene_var(hor_mean))
        
        
    def step(self):
        
        data = self.collect_data()
        #print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        if self.schedule.steps % 100 == 0: 
            self.data_file.flush()
        super().step()
        

In [ ]:
import sys

#0.01, 0.005, 0.001
sys.argv = 'm6 0.01 0.01 3'.split(' ')

muta_sd, env_step, lifetime = float(sys.argv[1]), float(sys.argv[2]), float(sys.argv[3])

fixed_params = {'N':1000, 'width':50, 'height':50}

variable_params = {'specific_kargs': [{'lifetime': lifetime, 'gene_sd':muta_sd, 'env_step':env_step}]}

batch_run = BatchRunnerMP(
    M_6, 1,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=1,
    max_steps=5000,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./res/M5/lifetime_3.0_gene_sd_0.01_env_step_0.01/82ce50ea-d9ae-11e9-a44f-f2189850a77c
0 population size:  1000
1 population size:  1493
2 population size:  1915
3 population size:  1529
4 population size:  1602
5 population size:  1709
6 population size:  1673
7 population size:  1679
8 population size:  1672
9 population size:  1641
10 population size:  1633
11 population size:  1639
12 population size:  1635
13 population size:  1631
14 population size:  1655
15 population size:  1667
16 population size:  1633
17 population size:  1635
18 population size:  1628
19 population size:  1630
20 population size:  1588
21 population size:  1577
22 population size:  1582
23 population size:  1602
24 population size:  1577
25 population size:  1542
26 population size:  1525
27 population size:  1519
28 population size:  1539
29 population size:  1497
30 population size:  1487
31 population size:  1532
32 population size:  1524
33 population size:  1525
34 population size:  1494


304 population size:  1653
305 population size:  1652
306 population size:  1652
307 population size:  1655
308 population size:  1654
309 population size:  1635
310 population size:  1625
311 population size:  1625
312 population size:  1599
313 population size:  1586
314 population size:  1588
315 population size:  1597
316 population size:  1584
317 population size:  1558
318 population size:  1551
319 population size:  1537
320 population size:  1524
321 population size:  1537
322 population size:  1561
323 population size:  1545
324 population size:  1527
325 population size:  1495
326 population size:  1523
327 population size:  1506
328 population size:  1475
329 population size:  1446
330 population size:  1438
331 population size:  1445
332 population size:  1413
333 population size:  1409
334 population size:  1363
335 population size:  1354
336 population size:  1306
337 population size:  1303
338 population size:  1255
339 population size:  1209
340 population size:  1210
3

608 population size:  1618
609 population size:  1579
610 population size:  1557
611 population size:  1599
612 population size:  1591
613 population size:  1589
614 population size:  1571
615 population size:  1576
616 population size:  1545
617 population size:  1523
618 population size:  1528
619 population size:  1526
620 population size:  1456
621 population size:  1450
622 population size:  1435
623 population size:  1441
624 population size:  1469
625 population size:  1428
626 population size:  1400
627 population size:  1350
628 population size:  1354
629 population size:  1354
630 population size:  1366
631 population size:  1322
632 population size:  1302
633 population size:  1275
634 population size:  1230
635 population size:  1195
636 population size:  1160
637 population size:  1143
638 population size:  1110
639 population size:  1106
640 population size:  1145
641 population size:  1154
642 population size:  1194
643 population size:  1232
644 population size:  1252
6

912 population size:  1543
913 population size:  1550
914 population size:  1505
915 population size:  1483
916 population size:  1507
917 population size:  1493
918 population size:  1437
919 population size:  1401
920 population size:  1386
921 population size:  1401
922 population size:  1390
923 population size:  1373
924 population size:  1338
925 population size:  1319
926 population size:  1331
927 population size:  1284
928 population size:  1251
929 population size:  1228
930 population size:  1202
931 population size:  1154
932 population size:  1132
933 population size:  1139
934 population size:  1152
935 population size:  1174
936 population size:  1146
937 population size:  1161
938 population size:  1210
939 population size:  1252
940 population size:  1299
941 population size:  1338
942 population size:  1378
943 population size:  1399
944 population size:  1415
945 population size:  1427
946 population size:  1473
947 population size:  1483
948 population size:  1468
9

...

2876 population size:  1494
2877 population size:  1474
2878 population size:  1460
2879 population size:  1448
2880 population size:  1460
2881 population size:  1429
2882 population size:  1415
2883 population size:  1412
2884 population size:  1367
2885 population size:  1417
2886 population size:  1372
2887 population size:  1333
2888 population size:  1285
2889 population size:  1232
2890 population size:  1220
2891 population size:  1234
2892 population size:  1240
2893 population size:  1233
2894 population size:  1254
2895 population size:  1268
2896 population size:  1257
2897 population size:  1256
2898 population size:  1257
2899 population size:  1300
2900 population size:  1340
2901 population size:  1331
2902 population size:  1366
2903 population size:  1366
2904 population size:  1404
2905 population size:  1443
2906 population size:  1464
2907 population size:  1477
2908 population size:  1477
2909 population size:  1503
2910 population size:  1520
2911 population size

3169 population size:  1407
3170 population size:  1408
3171 population size:  1382
3172 population size:  1364
3173 population size:  1354
3174 population size:  1368
3175 population size:  1339
3176 population size:  1305
3177 population size:  1300
3178 population size:  1286
3179 population size:  1277
3180 population size:  1240
3181 population size:  1198
3182 population size:  1173
3183 population size:  1124
3184 population size:  1109
3185 population size:  1085
3186 population size:  1053
3187 population size:  1042
3188 population size:  1050
3189 population size:  1080
3190 population size:  1097
3191 population size:  1129
3192 population size:  1164
3193 population size:  1145
3194 population size:  1170
3195 population size:  1230
3196 population size:  1278
3197 population size:  1278
3198 population size:  1294
3199 population size:  1299
3200 population size:  1360
3201 population size:  1408
3202 population size:  1428
3203 population size:  1463
3204 population size

3462 population size:  1522
3463 population size:  1483
3464 population size:  1475
3465 population size:  1474
3466 population size:  1496
3467 population size:  1465
3468 population size:  1411
3469 population size:  1392
3470 population size:  1393
3471 population size:  1368
3472 population size:  1320
3473 population size:  1288
3474 population size:  1327
3475 population size:  1311
3476 population size:  1297
3477 population size:  1285
3478 population size:  1260
3479 population size:  1235
3480 population size:  1164
3481 population size:  1203
3482 population size:  1205
3483 population size:  1198
3484 population size:  1268
3485 population size:  1269
3486 population size:  1307
3487 population size:  1310
3488 population size:  1316
3489 population size:  1329
3490 population size:  1349
3491 population size:  1352
3492 population size:  1384
3493 population size:  1389
3494 population size:  1389
3495 population size:  1430
3496 population size:  1463
3497 population size

3755 population size:  1483
3756 population size:  1472
3757 population size:  1436
3758 population size:  1440
3759 population size:  1448
3760 population size:  1403
3761 population size:  1367
3762 population size:  1391
3763 population size:  1351
3764 population size:  1324
3765 population size:  1275
3766 population size:  1271
3767 population size:  1262
3768 population size:  1203
3769 population size:  1157
3770 population size:  1112
3771 population size:  1101
3772 population size:  1052
3773 population size:  1030
3774 population size:  1030
3775 population size:  1065
3776 population size:  1082
3777 population size:  1122
3778 population size:  1161
3779 population size:  1169
3780 population size:  1207
3781 population size:  1191
3782 population size:  1214
3783 population size:  1269
3784 population size:  1289
3785 population size:  1323
3786 population size:  1344
3787 population size:  1378
3788 population size:  1427
3789 population size:  1445
3790 population size

4048 population size:  1522
4049 population size:  1554
4050 population size:  1538
4051 population size:  1490
4052 population size:  1460
4053 population size:  1447
4054 population size:  1435
4055 population size:  1429
4056 population size:  1432
4057 population size:  1426
4058 population size:  1413
4059 population size:  1389
4060 population size:  1355
4061 population size:  1362
4062 population size:  1336
4063 population size:  1302
4064 population size:  1263
4065 population size:  1270
4066 population size:  1231
4067 population size:  1226
4068 population size:  1178
4069 population size:  1201
4070 population size:  1193
4071 population size:  1244
4072 population size:  1267
4073 population size:  1252
4074 population size:  1266
4075 population size:  1276
4076 population size:  1335
4077 population size:  1341
4078 population size:  1349
4079 population size:  1377
4080 population size:  1404
4081 population size:  1442
4082 population size:  1469
4083 population size

In [3]:
fluctuating_env(0.1)

[0.5,
 0.6,
 0.7,
 0.7999999999999999,
 0.8999999999999999,
 0.9999999999999999,
 0.8999999999999999,
 0.7999999999999999,
 0.7,
 0.6,
 0.5,
 0.4,
 0.30000000000000004,
 0.20000000000000004,
 0.10000000000000003,
 2.7755575615628914e-17,
 0.10000000000000003,
 0.20000000000000004,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7,
 0.7999999999999999,
 0.8999999999999999,
 0.9999999999999999,
 0.8999999999999999,
 0.7999999999999999,
 0.7,
 0.6,
 0.5,
 0.4,
 0.30000000000000004,
 0.20000000000000004,
 0.10000000000000003,
 2.7755575615628914e-17,
 0.10000000000000003,
 0.20000000000000004,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7,
 0.7999999999999999,
 0.8999999999999999,
 0.9999999999999999,
 0.8999999999999999,
 0.7999999999999999,
 0.7,
 0.6,
 0.5,
 0.4,
 0.30000000000000004,
 0.20000000000000004,
 0.10000000000000003,
 2.7755575615628914e-17,
 0.10000000000000003,
 0.20000000000000004,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7,
 0.7999999999999999,
 0.8999999999999999,
 0.9999999